In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob as tb
import langid
import os
import re
import nltk
from nltk.stem import RSLPStemmer
from googletrans import Translator
import time
import random

In [2]:
def orengo(song):
    st = RSLPStemmer()
    words = filter(lambda x: len(x)>0, song)
    word_list = list(map(lambda x: st.stem(x), words))
    return word_list

In [3]:
def remove_stopwords(doc):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    new_doc = list(filter(lambda x: x not in stopwords, doc))
    return new_doc

In [4]:
df = pd.read_csv('.\data\dataset_lyrics.csv')

In [5]:
#removes compilations
df = df[(df.artist != '/colecao-amo-voce/')&(df.artist != '/corinhos-infantis/')&(df.artist != '/corinhos-evangelicos/')&
        (df.artist != '/harpa-crista/')&(df.artist != '/hinos/')&(df.artist != '/musicas-catolicas/')&(df.artist != '/musicas-gospel/')&
        (df.artist != '/pineapple/')&(df.artist != '/umbanda/')&(df.artist != '/alok/')]

In [6]:
#removes artists with less than 25 songs
df = df[(df.artist != '/melim/')&(df.artist != '/kell-smith/')&(df.artist != '/midiam-lima/')&(df.artist != '/isadora-pompeo/')]

In [7]:
#data cleansing
df['lyrics'] = df['lyrics'].str.strip('<div data-plugin=""googleTranslate"" id=""lyrics"">')
df['lyrics'] = df['lyrics'].str.strip('<img alt="Instrumental" class="instrumental-icon" src="/img/etc/instrumental.png"/')
df['lyrics'] = df['lyrics'].apply(lambda x: x.lower())

In [8]:
#df['lyrics'] = df['lyrics'].str.replace('\n', ' ')
df['lyrics'] = df['lyrics'].str.replace('\r', '')
df['lyrics'] = df['lyrics'].apply(lambda x: re.sub('[0-9!,.;:\]?}{()"["|@#$%*]', "", x))

In [9]:
dicionario_addr = 'dicionario1.txt'
dicionario = {}
t = Translator()

# translate phrase words to pt
# avoid due to excedent use of google requisitions (limited to 1000/day)
def get_pt(frase, vizinhanca):
    fraseretorno = ""
    split_frase = frase.split(" ")
    for x in range (0, len(split_frase)):
        palavra = split_frase[x]
        if(len(palavra)>2):                 #function only works for words with len>2
            if(check_isLanguage(palavra)):      #checks if word and phrase are same language
                fraseretorno+=palavra
            else:                           #not same language
                fraseretorno+=getFinalword(split_frase,x,vizinhanca)
        else:
            fraseretorno+=palavra
        fraseretorno+=" "
    return fraseretorno

def check_isLanguage(palavra):
    lang_palavra = t.detect(palavra).lang
    if(lang_palavra != "pt"):
        return False
    return True

def getFinalword(vetor_frase, palavra_position, neighborhood_size):       #check with neighborhood and gives final word
    menor, maior = calculo_neighborhood(vetor_frase,palavra_position, neighborhood_size)
    frasefinal = ""
    for i in range (menor, maior+1):
        frasefinal+= " " + vetor_frase[palavra_position+i]
    #frasefinaltb = tb(frasefinal)
    lang_frasefinal = t.detect(frasefinal).lang
    if(lang_frasefinal != "pt"):
        try:
            return pt_dictionary(vetor_frase[palavra_position], lang_frasefinal)
            #palavratb = tb(vetor_frase[palavra_position])
            #return str(palavratb.translate(to="pt"))
        except:
            #print('deu ruim)
            return vetor_frase[palavra_position]
    return vetor_frase[palavra_position]


def pt_dictionary(palavra, lang):
    if palavra in dicionario:
        print(palavra + ": já está no dic!")
        return dicionario[palavra]
    else:
        #palavratb = tb(palavra)
        #print("traduzir para colocar no dic " + palavra)
        traducao = t.translate(palavra, src=lang, dest='pt').text #str(palavratb.translate(from_lang=lang,to="pt"))
        #print("traduzido " + palavra + " para " + traducao)
        dicionario[palavra] = traducao
        add_dicionario(dicionario_addr,palavra)
        print(palavra + ": adicionado ao dic!")
        return traducao

def calculo_neighborhood(vetor_frase, palavra_position,neighborhood_size):
    menor = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position-i >= 0:
            menor = -i
            break
    maior = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position+i < len(vetor_frase):
            maior = i
            break
    return menor, maior

def load_dicionario(arquivo):
    if os.path.exists(arquivo):
        fl = open(arquivo, "r", encoding='utf-8')
        for line in fl.readlines():
            palavra, traducao = line.split(':')
            dicionario[palavra] = traducao
        fl.close()
    else:
        fl = open(arquivo, 'w+')

def add_dicionario(arquivo, chave):
    if os.path.exists(arquivo):
        f = open(arquivo,'a', encoding='utf-8')
    else:
        f = open(arquivo, 'w+',encoding='utf-8')
    f.write(chave +":"+ dicionario[chave]+"\n")
    f.close()


In [10]:
#google for translation only


def get_pt2(frase, vizinhanca): #vizinhanca defines the context size for the word language analysis
    fraseretorno = ""
    split_frase = frase.split(" ")
    for x in range (0, len(split_frase)):
        palavra = split_frase[x].lstrip().rstrip()
        if(len(palavra)<2):
            fraseretorno+=palavra
        else:
            if(check_isLanguage2(palavra)):      #checks if word is in portuguese
                fraseretorno+=palavra
            else:                           #not pt
                fraseretorno+=getFinalword(split_frase,x,vizinhanca)
            fraseretorno+=" "
    return fraseretorno

def check_isLanguage2(palavra):
    lang_palavra = langid.classify(palavra)
    if(lang_palavra != "pt"):
        return False
    return True

def getFinalword(vetor_frase, palavra_position, neighborhood_size):       #check with neighborhood and gives final word
    menor, maior = calculo_neighborhood(vetor_frase,palavra_position, neighborhood_size)
    frasefinal = ""
    for i in range (menor, maior+1):
        frasefinal+= " " + vetor_frase[palavra_position+i]
    lang_frasefinal = langid.classify(frasefinal)
    if(lang_frasefinal[0] != "pt"):
        return pt_dictionary2(vetor_frase[palavra_position], lang_frasefinal)
    return vetor_frase[palavra_position]

def calculo_neighborhood(vetor_frase, palavra_position,neighborhood_size):
    menor = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position-i >= 0:
            menor = -i
            break
    maior = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position+i < len(vetor_frase):
            maior = i
            break
    return menor, maior

def pt_dictionary2(palavra, lang):
    if palavra in dicionario:
        print(palavra + ": já está no dic!")
        return dicionario[palavra]
    else:
        palavratb = tb(palavra)
        print(palavratb)
        print("traduzir para colocar no dic " + str(palavra) + " de " + lang[0])
        traducao = palavra
        try:
            traducao = str(palavratb.translate(from_lang=lang[0],to="pt"))
            print("traduzido " + palavra + " para " + traducao)
            dicionario[palavra] = traducao
            add_dicionario(dicionario_addr,palavra)
            print(palavra + ": adicionado ao dic!")
            return traducao
        except Exception as ex:
            if(type(ex).__name__ != "NotTranslated"):
                print(palavra + " Exception: " + type(ex).__name__ )
                return None
            else:
                print("traduzido " + palavra + " para " + traducao)
                dicionario[palavra] = traducao
                add_dicionario(dicionario_addr,palavra)
                print(palavra + ": adicionado ao dic!")
                return traducao
                #se nao foi erro em que traducao igual a palavra recebida
        #traducao = t.translate(palavra, src=lang, dest='pt').text.rstrip().lstrip()
        #traducao = "teste"
        
    
def load_dicionario(arquivo):
    if os.path.exists(arquivo):
        fl = open(arquivo, "r", encoding='utf-8')
        for line in fl.readlines():
            print(line)
            palavra, traducao = line.split(':')
            dicionario[palavra] = traducao
        fl.close()
    else:
        fl = open(arquivo, 'w+')

def add_dicionario(arquivo, chave):
    if os.path.exists(arquivo):
        f = open(arquivo,'a', encoding='utf-8')
    else:
        f = open(arquivo, 'w+',encoding='utf-8')
    f.write(chave +":"+ dicionario[chave]+"\n")
    f.close()
    
def traduzindo(song):
    verses = song.split('\n')
    full_song = ""
    #print(verses)
    for v in verses:
        #print (v)
        wait = random.randint(1,6)
        translated_verses = get_pt2(v,1)
        full_song+=translated_verses
    #print("fullsong:" + full_song)
    time.sleep(wait)
    return full_song

df['lyrics'] = df['lyrics'].str.replace('\n\n', ' \n ')


dicionario_addr = 'dicionario_teste.txt' 
dicionario = {}
t = Translator()
langid.set_languages(['pt','es','en'])
load_dicionario(dicionario_addr)
#for d in dicionario:
#    print (d + " -> " + dicionario[d])

#palavratb = tb("te")
#trad = str(palavratb.translate(from_lang="en",to="pt"))

#print(trad)

for index, row in df.iterrows():
    #print(row['lyrics'])
    row['translated'] = traduzindo(row['lyrics'])
    time.sleep(2)
    print("tradução: " + row['translated'] + "\n\n" )
    

te:você

faz:faz

bem:bem

mas:mas

coragem:coragem

me:mim

tocar:toque

amei:amei

agora:agora

vou:vou

que:que

bater:bater

cara:cara

entendedor:entendedor

garrafa:garrafa

precisa:preciso

mesa:mesa

de:de

mim:mim

igual:igual

dele:dar

atrás:de volta

pisa:pisa

se:Eu sei

por:por

ele:ele

pausa:Pausa

quem:queimar

amar:amar

sofrer:sofrer

quer:quer

sozinho:sozinho

vai:ir

sof:sof

só:somente

ficou:ficou

na:na

forca:forca

vida:vida

desandou:desandou

com:com

faltou:faltou

bebi:bebi

era:foi

porque:porque

ela:o a

chegou:chegou

pra:pra

metade:metade

amo:eu amo

da:dá

ter:ter

tá:tá

ver:ver

sinto:sinto

culpada:culpado

ligar:paquerar

falo:falo

sim:sim

pior:pior

desconta:desconto

horas:horas

aí:lá

boca:boca

anda:e

dentro:dentro

realmente:realmente

esquecer:esqueleto

pedi:pedia

perder:perder

logo:logótipo

sabia:sábio

escutar:escutar

sorte:desenhar

errar:errar

fez:fez

mudo:mudo

naufragou:naufragou

erros:erros

nunca:nunca

calma:acalme-s

mas: já está no dic!
mas: já está no dic!
amei: já está no dic!
me: já está no dic!
agora: já está no dic!
vou: já está no dic!
que: já está no dic!
bater: já está no dic!
que: já está no dic!
cara: já está no dic!
entendedor: já está no dic!
tradução: sei bem oque te
 faz
 bem
 eu sei mas
 no fundo eu já tentei não faltou coragem
 éuma hora eu ia mim
 toque
 que você não vai mais voltar não receber mensagem também émensagem sei que opra sempre virou pó ena cabeça deu um nó mas
 eu tô bem consciente mas
 amei
 amei sozinho mas por dois mim
 conformei que agora
 enão depois vou
 ter que seguir em frente preocupa não que
 eu não vou bater
 no seu portão preocupa não que não vai ver mais omeu nome nenhuma ligação preocupa não que
 eu vou tomar vergonha na cara
 preocupa não pra um bom entendedor
 meia ausência b


garrafa: já está no dic!
precisa: já está no dic!
mesa: já está no dic!
precisa: já está no dic!
de: já está no dic!
mim: já está no dic!
igual: já está no dic!
dele: já está no

KeyboardInterrupt: 

In [12]:
#counting english and spanish words

langid.set_languages(['pt', 'en', 'es'])

def get_pt_only(frase, vizinhanca): #vizinhanca defines the context size for the word language analysis
    #print(frase)
    counter = 0
    fraseretorno = ""
    split_frase = frase.split(" ")
    for x in range (0, len(split_frase)):
        palavra = split_frase[x].lstrip().rstrip()
        #print (palavra)
        if(check_isLanguage_pt(palavra)):      #checks if word is in portuguese
            fraseretorno+=palavra + " "
        else:
            palavra_final=getFinallang(split_frase,x,vizinhanca)
            if(palavra_final == palavra):
                fraseretorno+=palavra + " "
            else:
                counter+=1
    return fraseretorno, counter

def check_isLanguage_pt(palavra):
    lang_palavra = langid.classify(palavra)
    #print(palavra + " " + lang_palavra[0])
    if(lang_palavra[0] != "pt"):
        return False
    return True

def getFinallang(vetor_frase, palavra_position, neighborhood_size):       #check with neighborhood and gives final word
    menor, maior = calculo_neighborhood(vetor_frase,palavra_position, neighborhood_size)
    frasefinal = ""
    for i in range (menor, maior+1):
        frasefinal+= " " + vetor_frase[palavra_position+i]
    lang_frasefinal = langid.classify(frasefinal)
    if(lang_frasefinal[0] != "pt"):
        return ""
    else:
        return vetor_frase[palavra_position]
    
def calculo_neighborhood(vetor_frase, palavra_position,neighborhood_size):
    menor = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position-i >= 0:
            menor = -i
            break
    maior = 0
    for i in range (neighborhood_size,0,-1):
        if palavra_position+i < len(vetor_frase):
            maior = i
            break
    return menor, maior
    return vetor_frase[palavra_position]


def toPT(song):
    import time
    verses = song.split('\n')
    contador = 0
    full_song = ""
    #print(verses)
    for v in verses:
        #print (v)
        translated_verses, foreign = get_pt_only(v,3)
        full_song+=translated_verses
        contador+=foreign
    #print("fullsong:" + full_song)
    time.sleep(1)
    return full_song, contador

for index, row in df.iterrows():
    print(row['lyrics'])
    row['translated'], row['foreign'] = toPT(row['lyrics']) 
    print("tradução: " + row['translated'] + "\nforeign: " + str(row['foreign']) + "\n\n" )

sei bem
o que te faz bem eu sei
mas no fundo eu já tentei não faltou coragem
é uma hora eu ia me tocar que você não vai mais voltar
não receber mensagem também é mensagem
sei que o pra sempre virou pó
e na cabeça deu um nó mas eu tô bem consciente
mas amei
amei sozinho mas por dois
me conformei que agora e não depois
vou ter que seguir em frente
preocupa não
que eu não vou bater no seu portão
preocupa não
que não vai ver mais o meu nome nenhuma ligação
preocupa não
que eu vou tomar vergonha na cara
preocupa não
pra um bom entendedor
meia ausência b
tradução: sei bem o que te faz bem eu sei mas no fundo eu já tentei não faltou coragem é uma hora eu ia me tocar que você não vai mais voltar não receber mensagem também é mensagem sei que o pra sempre virou pó e na cabeça deu um nó mas eu tô bem consciente amei sozinho mas por dois me conformei que agora e não depois vou ter que seguir em frente preocupa não que eu não vou bater no seu portão preocupa não que não vai ver mais o meu nome nen

KeyboardInterrupt: 

In [11]:
def translate(song):
    import time
    verses = song.split('\n')
    translated_verses = list(map(lambda x: get_pt(x,1), verses))
    full_song = "".join(translated_verses)
    print(full_song)
    time.sleep(1)
    return full_song

In [14]:
#df['translated'] = df['lyrics'].apply(lambda x: translate(x))
for index, row in df.iterrows():
    row['translated'] = translate(row['lyrics'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
df[df['artist'] == '/roberto-carlos/']

,artist,music_name,lyrics
50,/roberto-carlos/,/roberto-carlos/como-e-grande-o-meu-amor-por-v...,eu tenho tanto pra lhe falar\nmas com palavras...
51,/roberto-carlos/,/roberto-carlos/amigo-letras.html,você meu amigo de fé meu irmão camarada\namigo...
52,/roberto-carlos/,/roberto-carlos/nossa-senhora-2.html,cubra-me com seu manto de amor\nguarda-me na p...
53,/roberto-carlos/,/roberto-carlos/amor-sem-limite.html,quando a gente ama alguém de verdade\nesse amo...
54,/roberto-carlos/,/roberto-carlos/outra-vez.html,você foi o maior dos meus casos\nde todos os a...
55,/roberto-carlos/,/roberto-carlos/emocoes-letras.html,quando eu estou aqui\neu vivo esse momento lin...
56,/roberto-carlos/,/roberto-carlos/detalhes.html,não adianta nem tentar me esquecer\ndurante mu...
57,/roberto-carlos/,/roberto-carlos/lady-laura-letras.html,ho às vezes vontade de ser\nnovamente um menin...
58,/roberto-carlos/,/roberto-carlos/meu-querido-meu-velho-meu-amig...,esses seus cabelos brancos bonitos\nesse olhar...
59,/roberto-carlos/,/roberto-carlos/cama-e-mesa.html,eu quero ser sua canção eu quero ser seu tom\n...


In [17]:
df.iloc[70,2]

'o conta gota da saudade tá pingando\nde pouco a pouco gota a gota vou tirando\nvocê de mim você de mim\nconfesso que isso nunca teve nos meus planos\neu não to bem mas até que to me virando\nmelhor assim melhor assim\ne vai ter noite que eu vou dormir com vontade\nvou digitar mas não vou enviar mensagem\ne pouco a pouco vou me afastando de você\ne de passo em passo vou te esquecer\npedacinho por pedaço eu\narranco você do coração você do coração\ne de passo em passo vou te esquecer\npedacinho por pedaço eu\narranco você da minha vida cancela a recaída\ne vai noite que eu vou dormir com vontade\nvou digitar mas não vou enviar mensagem\npouco a pouco vou me afastando de você\ne de passo em passo vou te esquecer\npedacinho por pedaço eu\narranco você do coração você do coração\ne de passo em passo vou te esquecer\npedacinho por pedaço eu\narranco você da minha vida cancela a recaí'

In [24]:
#creates new df where the column lyrics carries every song of a given artist
df1 = df.groupby('artist')['lyrics'].apply(' '.join).reset_index()

In [11]:
#applies stemmer and creates new column
df1['orengo'] = df1['lyrics'].apply(lambda x: orengo(x.split(' ')))

In [12]:
#removes stopwords and creates new column
df1['no_stopwords'] = df1['lyrics'].apply(lambda x: remove_stopwords(x.split(' ')))

In [ ]:
#removes stopwords from translated text and creates new column
df1['ns_pt'] = df1['translated'].apply(lambda x: remove_stopwords(x.split(' ')))

In [13]:
#vocabulary
df1['unique'] = df1['lyrics'].apply(lambda x: len(set(x.split(' '))))

In [14]:
#vocabulary after stemming
df1['orengo_unique'] = df1['orengo'].apply(lambda x: len(set(x)))

In [15]:
#vocabulary without considering stopwords
df1['ns_unique'] = df1['no_stopwords'].apply(lambda x: len(set(x)))

In [16]:
#vocabulary without considering stopwords after stemming
df1['ns_orengo_unique'] = df1['no_stopwords'].apply(lambda x: len(set(orengo(x))))

In [ ]:
#vocabulary after translation
df1['unique_pt'] = df['translated'].apply(lambda x: len(set(x.split(' '))))

In [ ]:
#vocabulary after stemming + translation
df1['orengo_pt'] = df['translated'].apply(lambda x: len(set(x)))

In [ ]:
#vocabulary without considering stopwords (translated)
df1['ns_uni_pt'] = df1['ns_pt'].apply(lambda x: len(set(x)))

In [ ]:
#vocabulary without considering stopwords after stemming + translation
df1['ns_or_pt'] = df1['ns_pt'].apply(lambda x: len(set(orengo(x))))

In [17]:
df1.head()

,artist,lyrics,orengo,no_stopwords,unique,orengo_unique,ns_unique,ns_orengo_unique
0,/aline-barros/,mestre eu preciso de um milagre transforma min...,"[mestr, eu, precis, de, um, milagr, transform,...","[mestre, preciso, milagre, transforma, vida, e...",748,581,651,520
1,/amado-batista/,ao te ver pela primeira vez eu tremi todo uma ...,"[ao, te, ver, pel, prim, vez, eu, trem, tod, u...","[ver, primeira, vez, tremi, todo, coisa, tomou...",846,621,750,559
2,/ana-carolina/,eu e você não é assim tão complicado não é dif...,"[eu, e, voc, não, é, assim, tão, complic, não,...","[assim, tão, complicado, difícil, perceber, , ...",776,599,686,539
3,/anavitoria/,eu poderia acordar sem teu olhar de sono sem t...,"[eu, pod, acord, sem, teu, olh, de, son, sem, ...","[poderia, acordar, olhar, sono, lábio, dono, q...",852,673,767,613
4,/anderson-freire/,não consigo ir além do teu olhar tudo que eu c...,"[não, consig, ir, além, do, teu, olh, tud, que...","[consigo, ir, além, olhar, tudo, consigo, imag...",1127,868,1025,801


In [18]:
df1.to_csv('./data/final_data.csv', index=False)